In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import os
from PIL import Image

## Utility Function

In [5]:
def load_images_from_directory(directory):
    images = []
    image_paths = []
    for filename in os.listdir(directory):
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            image_path = os.path.join(directory, filename)
            img = load_img(image_path)
            img = img.resize((150, 150))
            img_array = img_to_array(img)
            images.append(img_array)
            image_paths.append(image_path)

    return np.array(images), np.array(image_paths)

## Data Augmentation

Preferrably done only for training data

Augment data collected to create more training data, including different picture rotation, zoom, flip, and brightness

In [6]:
base_folder = '/content/drive/MyDrive/datasetcollection/train/'
classes = ['green', 'red', 'yellow']

datagen = ImageDataGenerator(
    rotation_range=30,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=(0.5, 1.5)
)

for c in classes:
    class_path = os.path.join(base_folder, c)

    # Delete all previously modified images
    for fname in os.listdir(class_path):
        if fname.startswith('modimage'):
            os.remove(os.path.join(class_path, fname))

    # Load images after deleting old modified ones
    images, image_paths = load_images_from_directory(class_path)

    # Get 25% of images
    num_samples = int(len(images) *.25)
    random_indices = np.random.choice(len(images), size=num_samples, replace=False)
    sampled_images = images[random_indices]

    print(f"Augmenting {len(sampled_images)} images in class '{c}'...")

    for idx, image in enumerate(sampled_images):
        image = np.expand_dims(image, axis=0)  # Make it batch size 1
        aug_iter = datagen.flow(image, batch_size=1,
                                save_to_dir=class_path,
                                save_prefix='modimage',
                                save_format='jpg')
        for i in range(3):  # Create 3 augmentations per image
            next(aug_iter)


Augmenting 45 images in class 'green'...
Augmenting 57 images in class 'red'...
Augmenting 48 images in class 'yellow'...


## CNN Classifier

In [ ]:
def get_dataset(directory):
    classes = ['green', 'red', 'yellow']
    X = []
    y = []
    for c in classes:
        class_path = directory + c + "/"
        images, image_paths = load_images_from_directory(class_path)
        X.extend(images)
        y.extend([c] * len(images))
    return np.array(X), np.array(y)

In [ ]:
base_folder = '/content/drive/MyDrive/datasetcollection/'
X_train, y_train = get_dataset(base_folder + 'train/')
X_val, y_val = get_dataset(base_folder + 'val/')

In [ ]:
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((1038, 150, 150, 3), (277, 150, 150, 3), (1038,), (277,))

In [ ]:
X_train = X_train.astype('float32') / 255.0
X_val = X_val.astype('float32') / 255.0

encoder = OneHotEncoder(sparse_output=False)
y_train = encoder.fit_transform(y_train.reshape(-1, 1))
y_val = encoder.transform(y_val.reshape(-1, 1))

In [ ]:
model = Sequential()

# 1st Convolutional Block
model.add(Conv2D(32, (3, 3), padding='same', input_shape=(150, 150, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# 2nd Convolutional Block
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# 3rd Convolutional Block
model.add(Conv2D(128, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Fully Connected Layers
model.add(Flatten())
model.add(Dense(1024))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

# Output Layer (assuming 3 classes)
model.add(Dense(3))
model.add(Activation('softmax'))

# Compile the model
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(learning_rate=0.0001),
    metrics=['accuracy']
)

# only save best model, stop if no improvement
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)
checkpoint = ModelCheckpoint('/content/drive/My Drive/RoboCarV5_bestModel.keras', monitor='val_loss', save_best_only=True,mode='min', verbose=1 )


In [ ]:
model.fit(X_train, y_train, batch_size=4, epochs=12, validation_data=(X_val, y_val), callbacks=[checkpoint, early_stop])

Epoch 1/12
260/260 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7654 - loss: 0.6335
Epoch 1: val_loss improved from 1.04242 to 0.91555, saving model to /content/drive/My Drive/RoboCarV5_bestModel.keras
260/260 ━━━━━━━━━━━━━━━━━━━━ 458s 2s/step - accuracy: 0.7654 - loss: 0.6334 - val_accuracy: 0.6823 - val_loss: 0.9155
Epoch 2/12
260/260 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7453 - loss: 0.6291
Epoch 2: val_loss did not improve from 0.91555
260/260 ━━━━━━━━━━━━━━━━━━━━ 462s 2s/step - accuracy: 0.7453 - loss: 0.6290 - val_accuracy: 0.6679 - val_loss: 0.9915
Epoch 3/12
260/260 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7744 - loss: 0.5717
Epoch 3: val_loss did not improve from 0.91555
260/260 ━━━━━━━━━━━━━━━━━━━━ 450s 2s/step - accuracy: 0.7744 - loss: 0.5717 - val_accuracy: 0.6390 - val_loss: 1.1916
Epoch 4/12
260/260 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7887 - loss: 0.5749
Epoch 4: val_loss did not improve from 0.91555
260/260 ━━━━━━━━━━━━━━━━━━━━ 453s 2s/step - acc

In [ ]:
model.save('/content/drive/My Drive/RoboCarV4_6Epochs_NewRed.keras')

In [ ]:
img = load_img('/content/drive/MyDrive/datasetcollection/val/red/rpicam_image123.jpg')
img = img.resize((150, 150))
img_array = img_to_array(img)
img_array = np.array(img_array/255.0)
img_array = np.expand_dims(img_array, axis=0)
prediction = model.predict(img_array)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step


In [ ]:
print(encoder.inverse_transform(prediction))

NameError: name 'encoder' is not defined

## Optimize the Model for use on Pi

In [ ]:
import tensorflow as tf

# Load Keras model
model = tf.keras.models.load_model("/content/drive/MyDrive/RoboCarV5_bestModel.keras")

# Convert the model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]  # enables quantization
tflite_model = converter.convert()

# Save the quantized model
with open("/content/drive/My Drive/model_quant.tflite", "wb") as f:
    f.write(tflite_model)


Saved artifact at '/tmp/tmp7jr29ki0'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 150, 150, 3), dtype=tf.float32, name='input_layer_2')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  136699028357584: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136698941062608: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136698941061648: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136698941062416: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136698941055504: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136698941062992: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136698938491152: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136698938492688: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136698938492880: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136698938492112: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1366989384913